In [12]:
import torch
import torch.nn as neunet
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as npy

# Define Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 Dataset
trainingDatasetIP = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
Loader_train = torch.utils.data.DataLoader(trainingDatasetIP, batch_size=64, shuffle=True)

testingDatasetIP = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
Loader_test = torch.utils.data.DataLoader(testingDatasetIP, batch_size=64, shuffle=False)

# Define Cneunet Model
class Cneunet(neunet.Module):
    def __init__(self):
        super(Cneunet, self).__init__()
        self.convoL1 = neunet.Conv2d(3, 32, kernel_size=3, padding=1)
        self.convoL2 = neunet.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = neunet.MaxPool2d(kernel_size=2, stride=2)
        self.fulConLayer1 = neunet.Linear(64 * 8 * 8, 128)
        self.fulConLayer2 = neunet.Linear(128, 10)
        self.relu = neunet.ReLU()
        self.dropout = neunet.Dropout(0.5)

    def forward(self, ip):
        ip = self.pool(self.relu(self.convoL1(ip)))
        ip = self.pool(self.relu(self.convoL2(ip)))
        ip = torch.flatten(ip, 1)
        ip = self.dropout(self.relu(self.fulConLayer1(ip)))
        ip = self.fulConLayer2(ip)
        return ip

# Train and Evaluate the Model
def trainFunction(ipmodel, Loader_train, criterion, optimizer, epochCount=10):
    ipmodel.train()
    for indEpoch in range(epochCount):
        runningLoss = 0.0
        for images, labels in Loader_train:
            images, labels = images.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = ipmodel(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
        print(f'Epoch {indEpoch+1}, Loss: {runningLoss/len(Loader_train)}')

def evaluate_model(ipmodel, Loader_test):
    ipmodel.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for ipimages, labels in Loader_test:
            ipimages, labels = ipimages.cuda(), labels.cuda()
            outputs = ipmodel(ipimages)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Hyperparameters and Model Training
ipmodel = Cneunet().cuda()
criterion = neunet.CrossEntropyLoss()
optimizer = optim.Adam(ipmodel.parameters(), lr=0.001)

trainFunction(ipmodel, Loader_train, criterion, optimizer, epochCount=10)
evaluate_model(ipmodel, Loader_test)

# Fine-Tuning with ResNet
from torchvision import models

def fine_tune_resnet(Loader_train, Loader_test, epochCount=5):
    resnetOp = models.resnet18(pretrained=True)
    resnetOp.fc = nn.Linear(resnetOp.fc.in_features, 10)
    resnetOp = resnetOp.cuda()  # Ensure model is on GPU
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnetOp.parameters(), lr=0.0001)
    resnetOp.train()
    for indEpoch in range(epochCount):
        runningLoss = 0.0
        for ipimages, labels in Loader_train:
            ipimages, labels = ipimages.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = resnetOp(ipimages)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            runningLoss += loss.item()
        print(f'ResNet Epoch {indEpoch+1}, Loss: {runningLoss/len(Loader_train)}')
    evaluate_model(resnetOp, Loader_test)

fine_tune_resnet(Loader_train, Loader_test)


Epoch 1, Loss: 1.5406377531988236
Epoch 2, Loss: 1.213967132446406
Epoch 3, Loss: 1.0711401383895094
Epoch 4, Loss: 0.9823699260459227
Epoch 5, Loss: 0.9026068210449365
Epoch 6, Loss: 0.8414578674470677
Epoch 7, Loss: 0.7928262493570747
Epoch 8, Loss: 0.7449557831906297
Epoch 9, Loss: 0.7049689350454399
Epoch 10, Loss: 0.6709648398944484
Accuracy: 72.21%


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet Epoch 1, Loss: 0.97971738257524
ResNet Epoch 2, Loss: 0.5760679934793116
ResNet Epoch 3, Loss: 0.40526593072563794
ResNet Epoch 4, Loss: 0.29306672223846014
ResNet Epoch 5, Loss: 0.2176765161411613
Accuracy: 81.01%
